In [ ]:
# read electrode categories to see what proportion of electrodes are in SOZ 

In [112]:
import pandas as pd; pd.set_option('display.max_columns', 30)
import numpy as np
from cmlreaders import CMLReader, get_data_index
from ptsa.data.filters import ButterworthFilter
import sys
import os
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import *
from copy import copy
from scipy import stats
import pickle
plt.rcParams['pdf.fonttype'] = 42; plt.rcParams['ps.fonttype'] = 42 # fix fonts for Illustrator
sys.path.append('/home1/john/Downloads/ptsa_plot-master/')
sys.path.append('/home1/john/johnModules')
from brain_labels import MTL_labels, LTC_labels, PFC_labels, OTHER_labels, ALL_labels # all location labels
%load_ext autoreload
%autoreload
from general import *
from SWRmodule import *

df = get_data_index("r1") # all RAM subjects
exp = 'catFR1' # 'FR1' 'catFR1'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [113]:
# individual subs

subs = ['R1157C']# ['R1084T'] #['R1106M']

sub_df = df[(df.subject.isin(subs))]
# sub_df = df[(df.subject.isin(subs))  & (df.experiment == exp)] # all sessions for subs

# sub_df = sub_df[0:6]
sub_df

,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
891,NaN,NaN,protocols/r1/subjects/R1157C/localizations/0/m...,PS2,build,0,NaN,0,PS,1,protocols/r1/subjects/R1157C/localizations/0/m...,NaN,0,R1157C,R1157C,NaN,protocols/r1/subjects/R1157C/experiments/PS2/s...
892,NaN,NaN,protocols/r1/subjects/R1157C/localizations/0/m...,TH1,build,0,NaN,0,NaN,0,protocols/r1/subjects/R1157C/localizations/0/m...,NaN,0,R1157C,R1157C,NaN,protocols/r1/subjects/R1157C/experiments/TH1/s...
893,NaN,NaN,protocols/r1/subjects/R1157C/localizations/0/m...,TH1,build,0,NaN,0,NaN,1,protocols/r1/subjects/R1157C/localizations/0/m...,NaN,1,R1157C,R1157C,NaN,protocols/r1/subjects/R1157C/experiments/TH1/s...
894,NaN,NaN,protocols/r1/subjects/R1157C/localizations/0/m...,TH1,build,0,NaN,0,NaN,2,protocols/r1/subjects/R1157C/localizations/0/m...,NaN,2,R1157C,R1157C,NaN,protocols/r1/subjects/R1157C/experiments/TH1/s...
895,NaN,protocols/r1/subjects/R1157C/experiments/catFR...,protocols/r1/subjects/R1157C/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1157C/experiments/catFR...,0,NaN,0,protocols/r1/subjects/R1157C/localizations/0/m...,NaN,0,R1157C,R1157C,NaN,protocols/r1/subjects/R1157C/experiments/catFR...
896,NaN,protocols/r1/subjects/R1157C/experiments/catFR...,protocols/r1/subjects/R1157C/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1157C/experiments/catFR...,0,NaN,1,protocols/r1/subjects/R1157C/localizations/0/m...,NaN,1,R1157C,R1157C,NaN,protocols/r1/subjects/R1157C/experiments/catFR...
897,NaN,protocols/r1/subjects/R1157C/experiments/catFR...,protocols/r1/subjects/R1157C/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1157C/experiments/catFR...,0,NaN,2,protocols/r1/subjects/R1157C/localizations/0/m...,NaN,2,R1157C,R1157C,NaN,protocols/r1/subjects/R1157C/experiments/catFR...
898,NaN,protocols/r1/subjects/R1157C/experiments/catFR...,protocols/r1/subjects/R1157C/localizations/0/m...,catFR2,build,0,protocols/r1/subjects/R1157C/experiments/catFR...,0,NaN,0,protocols/r1/subjects/R1157C/localizations/0/m...,NaN,0,R1157C,R1157C,NaN,protocols/r1/subjects/R1157C/experiments/catFR...


In [103]:
# 575 FR sessions. first 18 of don't load so skip those 
if exp == 'FR1':
    exp_df = df[df.experiment==exp]
    exp_df = exp_df[
                    ((df.subject!='R1015J') | (df.session!=0)) & 
                    ((df.subject!='R1063C') | (df.session!=1)) & 
                    ((df.subject!='R1093J') | (~df.session.isin([1,2]))) &
                    ((df.subject!='R1100D') | (~df.session.isin([0,1,2]))) &
                    ((df.subject!='R1120E') | (df.session!=0)) &
                    ((df.subject!='R1122E') | (df.session!=2)) &
                    ((df.subject!='R1154D') | (df.session!=0)) &
                    ((df.subject!='R1186P') | (df.session!=0)) &
                    ((df.subject!='R1201P') | (~df.session.isin([0,1]))) &
                    ((df.subject!='R1216E') | (~df.session.isin([0,1,2]))) &
                    ((df.subject!='R1277J') | (df.session!=0)) &
                    ((df.subject!='R1413D') | (df.session!=0)) & 
                    ((df.subject!='R1123C') | (df.session!=2)) & # artifacts that bleed through channels (see SWR FR1 prob sessions ppt)
                    ((df.subject!='R1151E') | (~df.session.isin([1,2]))) & # more bleed-through artifacts (see same ppt)
                    ((df.subject!='R1275D') | (df.session!=3))  # 3rd session an actual repeat of 2nd session (Paul should have removed from database by now)
    #                 (df.subject!='R1065J') # sub with 9000 trials
                   ] 
elif exp == 'catFR1': 
    exp_df = df[df.experiment==exp]
    exp_df = exp_df[
                    ((df.subject!='R1044J') | (df.session!=0)) & # too few trials to do pg pairwise corr
                    ((df.subject!='R1491T') | (~df.session.isin([1,3,5]))) & # too few trials to do pg pairwise corr
                    ((df.subject!='R1486J') | (~df.session.isin([4,5,6,7]))) & # repeated data...will be removed at some point... @@
                    ((df.subject!='R1501J') | (~df.session.isin([0,1,2,3,4,5]))) & # these weren't catFR1 (and they don't load right anyway)
                    ((df.subject!='R1235E') | (df.session!=0)) & # split EEG filenames error...documented on Asana
                    ((df.subject!='R1310J') | (df.session!=1)) & # session 1 is just a repeat of session 0
                    ((df.subject!='R1239E') | (df.session!=0)) # some correlated noise (can see in catFR1 problem sessions ppt)
                   ]
elif exp == '':
    
    # if no experiment then just grab first task file for each subject
    
    unique_subs = np.unique(df.subject)
    first_sub_session_idxs = []
    for sb in unique_subs:
        temp_sub_df = df[(df.subject.isin([sb]))]
        
        # if there are FR or catFR1 sessions, make sure you grab one of those
        temp_exps = temp_sub_df.experiment
        FR_mask = [tp=='catFR1' or tp=='FR1' for tp in temp_exps] 
        if sum(FR_mask)>0:
            FR_idx = findInd(FR_mask)[0] # if an FR task take the first one
        else:
            FR_idx = 0 # if no FR task then just take first session
        first_sub_session_idxs.append(temp_sub_df.index[FR_idx])

    exp_df = df[(df.index.isin(first_sub_session_idxs))]
    
exp_df.shape
# exp_df = exp_df[257:] # for catFR1 this is R1385E-onwwards
# exp_df = exp_df[472:] # for FR1 this is R1385E-onwwards
# exp_df = exp_df[298:]
exp_df

(422, 17)

,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
0,NaN,protocols/r1/subjects/R1001P/experiments/FR1/s...,protocols/r1/subjects/R1001P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1001P/experiments/FR1/s...,0,NaN,0,protocols/r1/subjects/R1001P/localizations/0/m...,NaN,0,R1001P,R1001P,NaN,protocols/r1/subjects/R1001P/experiments/FR1/s...
9,NaN,protocols/r1/subjects/R1002P/experiments/FR1/s...,protocols/r1/subjects/R1002P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1002P/experiments/FR1/s...,0,NaN,0,protocols/r1/subjects/R1002P/localizations/0/m...,NaN,0,R1002P,R1002P,NaN,protocols/r1/subjects/R1002P/experiments/FR1/s...
17,NaN,protocols/r1/subjects/R1003P/experiments/FR1/s...,protocols/r1/subjects/R1003P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1003P/experiments/FR1/s...,0,NaN,0,protocols/r1/subjects/R1003P/localizations/0/m...,NaN,0,R1003P,R1003P,1.0,protocols/r1/subjects/R1003P/experiments/FR1/s...
24,NaN,protocols/r1/subjects/R1004D/experiments/catFR...,protocols/r1/subjects/R1004D/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1004D/experiments/catFR...,0,NaN,0,protocols/r1/subjects/R1004D/localizations/0/m...,NaN,0,R1004D,R1004D,NaN,protocols/r1/subjects/R1004D/experiments/catFR...
25,NaN,protocols/r1/subjects/R1006P/experiments/FR1/s...,protocols/r1/subjects/R1006P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1006P/experiments/FR1/s...,0,NaN,0,protocols/r1/subjects/R1006P/localizations/0/m...,NaN,0,R1006P,R1006P,NaN,protocols/r1/subjects/R1006P/experiments/FR1/s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2407,NaN,protocols/r1/subjects/R1528E/experiments/RepFR...,protocols/r1/subjects/R1528E/localizations/0/m...,RepFR1,build,0,NaN,0,NaN,NaN,protocols/r1/subjects/R1528E/localizations/0/m...,NaN,0,R1528E,R1528E,1.0,protocols/r1/subjects/R1528E/experiments/RepFR...
2409,NaN,protocols/r1/subjects/R1529T/experiments/DBOY1...,protocols/r1/subjects/R1529T/localizations/0/m...,DBOY1,build,0,NaN,0,NaN,NaN,protocols/r1/subjects/R1529T/localizations/0/m...,NaN,1,R1529T,R1529T,3.4,protocols/r1/subjects/R1529T/experiments/DBOY1...
2419,NaN,protocols/r1/subjects/R1530J/experiments/catFR...,protocols/r1/subjects/R1530J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1530J/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1530J/localizations/0/m...,NaN,0,R1530J,R1530J,3.4,protocols/r1/subjects/R1530J/experiments/catFR...
2428,NaN,protocols/r1/subjects/R1532T/experiments/catFR...,protocols/r1/subjects/R1532T/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1532T/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1532T/localizations/0/m...,NaN,0,R1532T,R1532T,3.4,protocols/r1/subjects/R1532T/experiments/catFR...


In [115]:
%autoreload
HPC_labels,ENT_labels,PHC_labels = getMTLregions(MTL_labels)

### PARAMS ###

remove_soz_ictal = False # False only removes bad electrodes and not soz/ictal

### END PARAMS ###
    
regions_LR = []
hospital_field = []
experiment_checked = []
sub_save = []

# things I want to know about SOZ:
has_L_SOZ = []
has_R_SOZ = []
has_SOZ = []
has_L_interictal = []
has_R_interictal = []
has_interictal = []

for row in sub_df.itertuples(): #sub_df.itertuples():   
    sub = row.subject; session = row.session; temp_exp = row.experiment
    mont = int(row.montage); loc = int(row.localization)
    reader = CMLReadDFRow(row)
    
    evs = reader.load('task_events')
    
    try:
        contacts = reader.load('contacts')
    except:
        print('No contacts for '+sub+', session '+str(session))
        continue
        
    pairs = reader.load('pairs')
    try:
        localizations = reader.load('localization')
    except:
        localizations = []
    tal_struct, bipolar_pairs, mpchans = get_bp_tal_struct(sub, montage=mont, localization=loc)
    elec_regions,atlas_type,pair_number,has_stein_das = get_elec_regions(localizations,pairs) 
    
    import ipdb; ipdb.set_trace()
    elec_names = [name[2] for name in tal_struct] 

    # get bad channel mask
    try:
        elec_cats = reader.load('electrode_categories') # this is cool
    except:
        if remove_soz_ictal == True:
            print('No electrode categories for '+sub+', session '+str(session))
            continue
        else:
            elec_cats = []

    ## determine if SOZ sites in L and/or R hemisphere ##
    
    # Note: probably don't have to look up L/R here for purposes of determining if SOZ or interictal
    # electrodes exist, but doing it this way ensure we have hemispheric information about seizurogenic
    # activity which we want to make sure can be found from these files
    
    if elec_cats['soz']==['NONE']:
        has_R_SOZ.append(-1)
        has_L_SOZ.append(-1)
        has_SOZ.append(-1)
    else:
        # for each soz site find the L or R in the string 
        soz_elecs = []
        for elec in elec_cats['soz']:
            elec_letter_idx = 0
            while (elec[elec_letter_idx]!='L') and (elec[elec_letter_idx]!='R') and (elec_letter_idx<len(elec)-1):
                elec_letter_idx+=1
            soz_elecs.append(elec[elec_letter_idx])
            
        # now have list of Ls or Rs for each elec...unless names don't have "L" or "R" in them (happens on occasion)
        # so get hemisphere from MNI coordinates
        for i_elec,elec in enumerate(elec_cats['soz']):
            if (soz_elecs[i_elec]!='L') and (soz_elecs[i_elec]!='R'):
                
                elec_idx = [i for i,contact in enumerate(contacts.label) if contact==elec]                
                
                # either tal or mni work. -X in both means L and +X means R
                if 'tal.x' in contacts.iloc[elec_idx]: # if tal coords exist
                    tal_x = contacts.iloc[elec_idx]['tal.x'].values
                    if len(tal_x)>0:
                        if tal_x > 0: # right!
                            soz_elecs[i_elec] = 'R'
                        elif tal_x < 0: # left!
                            soz_elecs[i_elec] = 'L'
                        else:
                            soz_elecs[i_elec] = ''
                    else: 
                        soz_elecs[i_elec] = ''
                elif 'mni.x' in contacts.iloc[elec_idx]: # if MNI coords exist
                    mni_x = contacts.iloc[elec_idx]['mni.x'].values
                    if len(mni_x)>0:
                        if mni_x > 0: # right!
                            soz_elecs[i_elec] = 'R'
                        elif mni_x < 0: # left!
                            soz_elecs[i_elec] = 'L'
                        else:
                            soz_elecs[i_elec] = ''
                    else:
                        soz_elecs[i_elec] = ''
                
        # was there SOZ in L or R hemispheres?
        if ('R' in soz_elecs):
            has_R_SOZ.append(1)
        else:
            has_R_SOZ.append(0)
        if ('L' in soz_elecs):
            has_L_SOZ.append(1)
        else:
            has_L_SOZ.append(0)
            
        # for google doc purposes:
        if ( (has_R_SOZ[-1]==1) or (has_L_SOZ[-1])==1 )==1:
            has_SOZ.append(1)
        else:
            has_SOZ.append(0)
            
            
    ## determine if *interictal* sites in L and/or R hemisphere ##
    
    if elec_cats['interictal']==['NONE']:
        has_R_interictal.append(-1)
        has_L_interictal.append(-1)
        has_interictal.append(-1)
    else:
        # for each soz site find the L or R in the string 
        interictal_elecs = []
        for elec in elec_cats['interictal']:
            elec_letter_idx = 0
            while (elec[elec_letter_idx]!='L') and (elec[elec_letter_idx]!='R') and (elec_letter_idx<len(elec)-1):
                elec_letter_idx+=1
            interictal_elecs.append(elec[elec_letter_idx])
            
        # now have list of Ls or Rs for each elec...unless names don't have "L" or "R" in them (happens on occasion)
        # so get hemisphere from MNI coordinates
        for i_elec,elec in enumerate(elec_cats['interictal']):
            if (interictal_elecs[i_elec]!='L') and (interictal_elecs[i_elec]!='R'):

                elec_idx = [i for i,contact in enumerate(contacts.label) if contact==elec]                
                
                if 'tal.x' in contacts.iloc[elec_idx]: # if tal coords exist
                    tal_x = contacts.iloc[elec_idx]['tal.x'].values
                    if len(tal_x)>0:
                        if tal_x > 0: # right!
                            interictal_elecs[i_elec] = 'R'
                        elif tal_x < 0: # left!
                            interictal_elecs[i_elec] = 'L'
                        else:
                            interictal_elecs[i_elec] = ''
                    else: 
                        interictal_elecs[i_elec] = ''
                elif 'mni.x' in contacts.iloc[elec_idx]: # if MNI coords exist
                    mni_x = contacts.iloc[elec_idx]['mni.x'].values
                    if len(mni_x)>0:
                        if mni_x > 0: # right!
                            interictal_elecs[i_elec] = 'R'
                        elif mni_x < 0: # left!
                            interictal_elecs[i_elec] = 'L'
                        else:
                            interictal_elecs[i_elec] = ''
                    else:
                        interictal_elecs[i_elec] = ''
                
        # was there interictal in L or R hemispheres?
        if ('R' in interictal_elecs):
            has_R_interictal.append(1)
        else:
            has_R_interictal.append(0)
        if ('L' in interictal_elecs):
            has_L_interictal.append(1)
        else:
            has_L_interictal.append(0)
            
        # for google doc purposes:
        if ( (has_R_interictal[-1]==1) or (has_L_interictal[-1])==1 )==1:
            has_interictal.append(1)
        else:
            has_interictal.append(0)
            
    # if you get to here, then can append to subjects/experiments/etc
    hospital_field.append(sub[-1])
    experiment_checked.append(temp_exp)
    sub_save.append(sub)
    

#     import ipdb; ipdb.set_trace()
            
    bad_bp_mask = getBadChannels(tal_struct,elec_cats,remove_soz_ictal)

    electrode_search_range = getElectrodeRanges(elec_regions,temp_exp,sub,session,mont)

    # how many L and R sites in our key regions?
    ca1_labels = [HPC_labels[label] for label in [0,5,14,17]]
    dg_labels = [HPC_labels[label] for label in [3,8,15,18]]

    temp = np.zeros(8)
    for channel in electrode_search_range:

        if (elec_regions[channel] in ca1_labels) & \
        (bad_bp_mask[channel] == 0): # don't use bad electrodes
            if tal_struct[channel][2][0] == 'L': # if L ca1
                temp[0]+=1
            elif tal_struct[channel][2][0] == 'R':
                temp[1]+=1

        if (elec_regions[channel] in dg_labels) & \
        (bad_bp_mask[channel] == 0): # don't use bad electrodes
            if tal_struct[channel][2][0] == 'L':
                temp[2]+=1
            elif tal_struct[channel][2][0] == 'R':
                temp[3]+=1

        if (elec_regions[channel] in ENT_labels) & \
        (bad_bp_mask[channel] == 0): # don't use bad electrodes
            if tal_struct[channel][2][0] == 'L': # if L ca1
                temp[4]+=1
            elif tal_struct[channel][2][0] == 'R':
                temp[5]+=1

        if (elec_regions[channel] in PHC_labels) & \
        (bad_bp_mask[channel] == 0): # don't use bad electrodes
            if tal_struct[channel][2][0] == 'L': # if L ca1
                temp[6]+=1
            elif tal_struct[channel][2][0] == 'R':
                temp[7]+=1

    regions_LR = superVstack(regions_LR,temp)

SOZ_interictal = np.vstack([np.array(has_SOZ),np.array(has_interictal)]).T

SOZ_LR = np.vstack([np.array(has_L_SOZ),np.array(has_R_SOZ)]).T


if exp == '':
    exp = 'all_exp'
# values
np.savetxt('../compiled/electrode_categories/'+exp+'_CA1-DG-ENT-PHC_electrodes.csv',np.asarray(regions_LR), delimiter=",")
np.savetxt('../compiled/electrode_categories/'+exp+'_SOZ_interictal.csv',np.asarray(SOZ_interictal), delimiter=",")
np.savetxt('../compiled/electrode_categories/'+exp+'_SOZ_LR.csv',np.asarray(SOZ_LR), delimiter=",")
# np.savetxt('../compiled/localization/'+exp+'_sessions.csv',np.asarray(exp_df.session), delimiter=",")

# strings
np.savetxt('../compiled/electrode_categories/'+exp+'_subs.csv',np.asarray(sub_save), delimiter=",",fmt='%s')
np.savetxt('../compiled/electrode_categories/'+exp+'_hospitals.csv',np.asarray(hospital_field), delimiter=",",fmt='%s')
np.savetxt('../compiled/electrode_categories/'+exp+'_experiments_checked.csv',np.asarray(experiment_checked), delimiter=",",fmt='%s')


/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1157C/localizations/0/montages/0/neuroradiology/current_processed/contacts.json
/data/eeg/R1157C/tal/R1157C_talLocs_database_monopol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1157C/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1157C/tal/R1157C_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data/readers/tal.py:43: FutureWarning: Lab-specific readers may be moved to the cmlreaders package (https://github.com/pennmem/cmlreaders)
  FutureWarning)


--Call--
> /home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/IPython/core/displayhook.py(252)__call__()
    251 
--> 252     def __call__(self, result=None):
    253         """Printing with history cache management.



ipdb>  u


> <ipython-input-115-6db4401e57b6>(44)<module>()
     43 
---> 44     import ipdb; ipdb.set_trace()
     45     elec_names = [name[2] for name in tal_struct]



ipdb>  l


...

['ST2-ST3', 'ST3-ST4', 'ST4-ST5', 'ST5-ST6', 'ST6-ST7', 'ST7-ST8', 'ST8-ST9', 'ST9-ST10', 'SP1-SP2', 'SP2-SP3', 'SP3-SP4', 'SP4-SP5', 'SP5-SP6', 'SP6-SP8', 'SP8-SP10', 'SP10-SP11', 'SP11-SP12', 'SP12-SP13', 'SP13-SP14', 'MP2-MP3', 'MP3-MP4', 'MP4-MP6', 'MP6-MP8', 'MP8-MP10', 'MP10-MP11', 'MP11-MP12', 'MP12-MP13', 'MP13-MP14', 'MP14-MP15', 'IP1-IP2', 'IP2-IP3', 'IP3-IP4', 'IP4-IP5', 'IP5-IP6', 'IP6-IP7', 'IP7-IP8', 'IP8-IP10', 'IP10-IP12', 'IP12-IP13', 'IP13-IP14', 'IP14-IP15', 'IP15-IP16', 'PP1-PP2', 'PP2-PP3', 'PP3-PP4', 'PP4-PP5', 'PP5-PP6', 'PP6-PP7', 'PP7-PP8', 'PP8-PP9', 'PP9-PP10', 'PP10-PP11', 'PP11-PP12', 'PP12-PP13', 'PP13-PP14']


ipdb>  elec_regions[59:80]


array(['right cerebral white matter',
       'right ttg transverse temporal gyrus',
       'right cerebral white matter', 'right cerebral white matter',
       'right ttg transverse temporal gyrus', 'right pt planum temporale',
       'right pt planum temporale', 'right pt planum temporale',
       'right pt planum temporale', 'right pcu precuneus',
       'right pcu precuneus', 'right pcu precuneus',
       'right cerebral white matter', 'right cerebral white matter',
       'superiorparietal', 'superiorparietal', 'superiorparietal',
       'superiorparietal', 'right smg supramarginal gyrus',
       'supramarginal', 'right pcu precuneus'], dtype='<U36')


ipdb>  q


BdbQuit: 

In [110]:
SOZ_LR = np.vstack([np.array(has_L_SOZ),np.array(has_R_SOZ)]).T
np.savetxt('../compiled/electrode_categories/'+exp+'_SOZ_LR.csv',np.asarray(SOZ_LR), delimiter=",")

## writing electrode_categories.txt (from Joey)

In [ ]:
import numpy as np
import pandas as pd
# read REDCap data
cat = pd.read_csv('REDCap_localization_12302020.csv')

# separate df by event type
resec = cat[cat['Event Name']=='Localization and Resection'].drop(columns=['Event Name', 'Study site'])
info = cat[cat['Event Name']=='Subject Consent & Surgery '][['Subject Number', 'Study site']]

# merge different REDCap "events" for the same subjects, so we have one row per subject
cat = pd.merge(resec, info, on='Subject Number', how='right')

# if empty, insert string "NONE"
cat = cat.replace(np.nan, 'NONE')

# dictionary lookup for generating subject codes of form R1###X
site_dict = {'University of Pennsylvania':'P', 'Dartmouth University':'D', 
             'Jefferson Hospital':'J', 'Emory University':'E', 'Mayo Clinic':'M', 
             'UT Southwestern':'T', 'Columbia University':'C', 'NINDS':'N', 'University of Washington':'W'}
cat['Subject Code'] = np.array(['R1']*len(cat)) + cat['Subject Number'].astype(str).str.zfill(3)+ cat['Study site'].apply(lambda x: site_dict[x])
cat = cat.drop(columns=['Subject Number', 'Study site'])

In [ ]:
for i, row in cat.iterrows():
    subject = row['Subject Code']
    # get lists of electrodes in each category
    ictal_onset = row[0].split('  ')
    interictal = row[1].split('  ')
    lesions = row[2].split('  ')
    broken = row[3].split('  ')
    # nice formatting with consistent section headers and spacing
    lines = [subject, '', 'Ictal Onset:'] + ictal_onset + ['', 'Interictal Spikes:'] + \
            interictal + ['', 'Brain Lesions:'] + lesions + ['', 'Broken Leads:'] + broken
    # add newlines
    lines = [x+'\n' for x in lines]
    # write to scratch (can choose another location of course)
#     with open('/scratch/jrudoler/{}_electrode_categories.txt'.format(subject), 'w') as f:
#         f.writelines(lines)